First installing required libraries

In [1]:
!pip install transformers
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

importing packages

In [12]:
import pandas as pd
import pickle
import tensorflow as tf
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features
from tensorflow.keras import callbacks
import pickle
import numpy as np
from tqdm.notebook import tqdm
import hazm
import re
from string import punctuation
from transformers import TextClassificationPipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Embedding , Conv1D, GlobalMaxPooling1D, Dropout, Activation
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

loading data


In [2]:
df = pd.read_excel('/content/drive/MyDrive/datasets/all_data.xlsx')
df=df.sample(frac=1)
df.head()

,Unnamed: 0,text,label
3943,593,\nبا سلام شوهر من قرص ترامادول مصرف می کرد حدو...,6
5039,2927,سلام.من یک سال و خورده ایه ازدواج کردم .مادرشو...,1
6653,2059,سلام میخواستم بگم نامزد من هیچ میل جنسی تداره ...,3
8165,1763,\nسلام مدت ۱۳سال است قرص لورازپام مصرف میکنم ه...,6
3064,5357,بسیاری از ما می خواهیم کودکمان کاملا درست و بد...,2


cleaning the data

In [3]:
df = df.drop('Unnamed: 0',axis=1)
df = df.dropna()
df.head()

,text,label
3943,\nبا سلام شوهر من قرص ترامادول مصرف می کرد حدو...,6
5039,سلام.من یک سال و خورده ایه ازدواج کردم .مادرشو...,1
6653,سلام میخواستم بگم نامزد من هیچ میل جنسی تداره ...,3
8165,\nسلام مدت ۱۳سال است قرص لورازپام مصرف میکنم ه...,6
3064,بسیاری از ما می خواهیم کودکمان کاملا درست و بد...,2


counting labels


In [4]:
df['label'].value_counts()

4    2072
6    1731
0    1638
3     823
2     728
1     581
5     386
Name: label, dtype: int64

let's balance the data around 1000 samples


In [5]:
df_minority1 = df[df.label == 1]
df_minority2 = df[df.label == 2]
df_minority3 = df[df.label == 3]
df_minority5 = df[df.label == 5]
df_minority_upsampled1 = resample(df_minority1,
                                 replace=True,  # sample with replacement
                                 n_samples=1000,  # to match majority class
                                 random_state=123)  # reproducible results
df_minority_upsampled5 = resample(df_minority5,
                                 replace=True,  # sample with replacement
                                 n_samples=1000,  # to match majority class
                                 random_state=123)

df_minority_upsampled2 = resample(df_minority2,
                                 replace=True,  # sample with replacement
                                 n_samples=1000,  # to match majority class
                                 random_state=123)  # reproducible results
df_minority_upsampled3 = resample(df_minority3,
                                 replace=True,  # sample with replacement
                                 n_samples=1000,  # to match majority class
                                 random_state=123)


df_upsampled = pd.concat([df[df.label==0].sample(1000), df_minority_upsampled1,
                          df_minority_upsampled2,
                          df_minority_upsampled3,
                          df[df.label==4].sample(1000),
                          df_minority_upsampled5,
                          df[df.label==6].sample(1000)])

df = df_upsampled
df['label'].value_counts()

0    1000
1    1000
2    1000
3    1000
4    1000
5    1000
6    1000
Name: label, dtype: int64

In [6]:
df = df.sample(frac=1)
df.head()

,text,label
5302,اگر ازدواج زن بزرگتر از مرد باشد چكار بايد كرد,4
4123,سلام.._x000D_\nمن یه دختر ۲۴ ساله م که امسال س...,0
7381,مغز ما به دنبال توجیه و جواب هست که کمتر به دن...,0
2486,\nسلام \nمن از شما خیلی متشکرم که با حوصله وقت...,6
1054,شما تابحال تجربه ی افسردگی رو داشتید افسردگی ی...,0


now let's process the data and clean it for nlp task

In [7]:
stop_words = pickle.load(open('/content/drive/MyDrive/datasets/text_stop_words.pkl','rb'))
stop_words

['میشم',
 'اینکه',
 'یا',
 '000',
 'همه',
 'حتی',
 'از',
 'سلام',
 'داره',
 'میخوام',
 'اما',
 'باید',
 'نمی',
 'اون',
 'یک',
 'شما',
 'نمیدونم',
 'می',
 'چند',
 'در',
 'پیش',
 'خیلی',
 'تو',
 'دوست',
 'اصلا',
 'چون',
 'هم',
 'd_',
 'ماه',
 'است',
 'وقتی',
 'شده',
 'با',
 'این',
 'کنم',
 'سال',
 'الان',
 'های',
 'ها',
 'ولی',
 'رو',
 'ساله',
 'هست',
 'زندگی',
 'به',
 'هر',
 'ندارم',
 'ما',
 'کرد',
 'هیچ',
 'یه',
 'چیکار',
 'کنید',
 'من',
 'شدم',
 'بهم',
 'کردم',
 'تا',
 'میشه',
 'برام',
 'که',
 'را',
 'میکنه',
 'هستم',
 'دارم',
 'میگه',
 'چه',
 'دیگه',
 'برای',
 'بعد']

In [8]:
lemmatizer = hazm.Lemmatizer()
normalizer = hazm.Normalizer()
def preprocessing(text):

    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c not in punctuation)
    text = re.sub('\n',' ', text)
    text = re.sub('xD',' ', text)
    text = re.sub('؟',' ', text)
    text = re.sub('،',' ', text)
    text = normalizer.normalize(text)
    text = lemmatizer.lemmatize(text)
    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

In [9]:
# apply the function to data
df['new_text'] = df['text'].apply(preprocessing)
df.head(4)

,text,label,new_text
5302,اگر ازدواج زن بزرگتر از مرد باشد چكار بايد كرد,4,اگر ازدواج زن بزرگتر مرد باشد چکار
4123,سلام.._x000D_\nمن یه دختر ۲۴ ساله م که امسال س...,0,دختر م امسال آخر دانشگاهمه پدرم معتاده و فهمید...
7381,مغز ما به دنبال توجیه و جواب هست که کمتر به دن...,0,مغز دنبال توجیه و جواب کمتر دنبال دلیل باشیم ا...
2486,\nسلام \nمن از شما خیلی متشکرم که با حوصله وقت...,6,متشکرم حوصله وقت خود اختیار مردم می‌گذارید آرش...


now label encoding the data

In [10]:
labelencoder = LabelEncoder()
y_cnn = labelencoder.fit_transform(df['label'])

y_cnn = np_utils.to_categorical(y_cnn, num_classes= 7)
y_cnn, len(y_cnn)

(array([[0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 7000)

Building CNN model

In [11]:
max_features = 216927
maxlen = 200
embedding_dims = 50

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features,
    standardize = None,
    ngrams = 2,
    output_mode='int',
    output_sequence_length=maxlen,

)
vectorize_layer.adapt(df.new_text)
vectorize_layer.vocabulary_size()

213603

In [18]:

def model1(x_train,y_train,classes,epoches,x_test,y_test,filters,EMBEDDING_DIM,kernel_size):
    hidden_dims = 250
    # CNN with max pooling imeplementation
    model = Sequential()
    model.add(tf.keras.Input(shape=(1,), dtype=tf.string,name='text_input'))
    model.add(vectorize_layer)
    model.add(Embedding(max_features,
                        EMBEDDING_DIM,
                        input_length=maxlen))
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))

    model.add(tf.keras.layers.GlobalMaxPool1D())
    model.add(Dense(hidden_dims,activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes , activation='softmax'))

    early_stoping = callbacks.EarlyStopping(
    min_delta = 0.1,
        patience=2,
        restore_best_weights=True
    )

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.RMSprop(
                        learning_rate=0.0009,
                      centered=True
                  ),
                  metrics=['accuracy']
                )

    model.fit(x_train, y_train,
              batch_size=64,
              epochs=epoches,
              validation_data=(x_test,y_test),
             callbacks=[early_stoping]
             )

    return model

spliting data to 2 part

In [14]:
x_train_cnn, x_test_cnn, Y_train_cnn, Y_test_cnn = train_test_split(df.new_text,y_cnn,stratify=y_cnn)

In [16]:
len(x_train_cnn), len(x_test_cnn), Y_train_cnn, Y_test_cnn

(5250,
 1750,
 array([[0., 0., 0., ..., 0., 1., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]], dtype=float32),
 array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.]], dtype=float32))

In [19]:
cnn_model = model1(x_train_cnn,Y_train_cnn,7,12,x_test_cnn,Y_test_cnn,filters=222,EMBEDDING_DIM=120,kernel_size=2)

Epoch 1/12
83/83 [==============================] - 40s 464ms/step - loss: 1.7559 - accuracy: 0.3293 - val_loss: 1.3612 - val_accuracy: 0.4331
Epoch 2/12
83/83 [==============================] - 36s 432ms/step - loss: 1.0436 - accuracy: 0.6514 - val_loss: 0.8259 - val_accuracy: 0.7217
Epoch 3/12
83/83 [==============================] - 35s 427ms/step - loss: 0.6296 - accuracy: 0.7962 - val_loss: 0.6144 - val_accuracy: 0.7926
Epoch 4/12
83/83 [==============================] - 35s 427ms/step - loss: 0.3863 - accuracy: 0.8897 - val_loss: 0.4979 - val_accuracy: 0.8349
Epoch 5/12
83/83 [==============================] - 34s 410ms/step - loss: 0.2046 - accuracy: 0.9484 - val_loss: 0.4324 - val_accuracy: 0.8554
Epoch 6/12
83/83 [==============================] - 34s 409ms/step - loss: 0.0975 - accuracy: 0.9785 - val_loss: 0.4212 - val_accuracy: 0.8651


now lets predict with model and then save it to use it later

In [20]:
cnn_model.predict(['سلام من احساس سردرد دارم'])

1/1 [==============================] - 0s 310ms/step


array([[0.6181535 , 0.03033816, 0.06429069, 0.03132221, 0.08011667,
        0.15407766, 0.02170099]], dtype=float32)

In [21]:
cnn_model.save('cnn_model')